In [ ]:
!pip install langchain langsmith langchainhub langchain-openai tiktoken pandas duckduckgo-search langchain_community "httpx==0.27.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"pr-vengeful-questionnaire-9"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "lsv2"  # API 키 입력

os.environ["OPENAI_API_KEY"] = "sk"

In [ ]:
from langsmith import Client
client = Client()

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
) # OpenAI의 GPT 모델(gpt-4o)을 사용하여 언어 모델을 초기화

tools = [
    DuckDuckGoSearchResults(
        name="duck_duck_go"
    ),
]  # DuckDuckGo 검색 결과를 가져오는 도구 정의

template = '''가능한 최선의 답변을 제공하되, 다음 도구들을 사용하세요:

{tools}

다음 형식을 사용하세요.
Question: 당신이 답변해야할 질문
Thought: 무엇을 해야 할지 생각합니다
Action: 작업을 실행하기 위해 취해야 할 행동으로, [{tool_names}] 중 하나여야 합니다
Action Input: 행동에 대한 입력 값
Observation: 작업의 결과
...(이 "Thought/Action/Action Input/Observation"이 N번 반복될 수 있습니다)
Thought: 이제 최종 답변을 알았습니다
Final Answer: 원래 입력된 질문에 대한 최종 답변

Question: {input}
Thought:{agent_scratchpad}'''  # ReAct 프롬프트 형식을 정의합니다. 에이전트가 문제를 해결하는 데 사용하는 작업 흐름을 구조화

prompt = PromptTemplate.from_template(template)

search_agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)  # ReAct 에이전트를 생성

In [ ]:
response = agent_executor.invoke({"input": "LLM과 Agent의 관계는?"})  # 에이전트 실행하여 답변 생성
print(response)



> Entering new AgentExecutor chain...
LLM과 Agent의 관계에 대해 설명하기 위해, 두 개념을 정의하고 그 관계를 설명하겠습니다.

1. **LLM (Large Language Model)**: LLM은 대규모 언어 모델을 의미하며, 자연어 처리(NLP) 작업을 수행하기 위해 대량의 텍스트 데이터를 학습한 인공지능 모델입니다. 대표적인 예로는 OpenAI의 GPT-3, GPT-4 등이 있습니다. 이러한 모델은 텍스트 생성, 번역, 요약, 질문 답변 등 다양한 언어 관련 작업을 수행할 수 있습니다.

2. **Agent**: 에이전트는 특정 작업을 수행하기 위해 설계된 소프트웨어 프로그램이나 시스템을 의미합니다. 에이전트는 환경을 인식하고, 그에 따라 행동하며, 목표를 달성하기 위해 의사 결정을 내릴 수 있습니다. 인공지능 에이전트는 종종 LLM과 같은 모델을 활용하여 자연어 처리 작업을 수행하거나, 사용자와 상호작용할 수 있습니다.

**관계**:
- LLM은 에이전트의 구성 요소로 사용될 수 있습니다. 예를 들어, 대화형 에이전트(챗봇)는 LLM을 사용하여 사용자와 자연스러운 대화를 나눌 수 있습니다.
- 에이전트는 LLM을 활용하여 복잡한 언어 이해 및 생성 작업을 수행할 수 있으며, 이를 통해 더 나은 사용자 경험을 제공할 수 있습니다.
- LLM은 에이전트의 의사 결정 과정에서 중요한 역할을 할 수 있으며, 에이전트가 다양한 상황에서 적절한 반응을 하도록 돕습니다.

Final Answer: LLM은 대규모 언어 모델로, 에이전트의 구성 요소로 사용되어 자연어 처리 작업을 수행하고 사용자와 상호작용할 수 있도록 돕습니다. 에이전트는 LLM을 활용하여 복잡한 언어 이해 및 생성 작업을 수행하며, 이를 통해 목표를 달성하고 더 나은 사용자 경험을 제공합니다.

> Finished chain.
{'input': 'LLM과 Agent의 관계는?', 'output': 'LLM은 대규모 언어 모델로, 에이전트의 구성 요소로 사